<img src="./venn_diagram.png" alt="Venn Diagram" title="Venn Diagram" width="600" height="600">

# Autograd
Autograd keeps a record of tensors on all executed operations in a DAG. By tracing this graph in a topological way you can automatically compute the gradients using the chain rule.
<br><br>

Pytorch is a Python and C++ library that utilizes autograd for large matmuls in neural networks.<br>
SheetJS is a javascript library for reading and writing data from spreadsheets

This demo goes over the results of autograd implemented in Python and Javascript and we will talk about
- General math overview
- Viability of Javascript for creating Pytorch/Pandas like libraries
- Performance differences

### Why was this project chosen?
This project was chosen because it is the intersection of
- Javascript
- Python
- Mathematics

Autograd libraries are complex projects that involve heavy use software engineering and mathematics. This project can help us explore how well suited each programming lanuguage is for data intensive operations.
<br><br><br><br><br><br>

### Math Involved
A majority of the math revolves around
1. Partial Differentiation
2. Gradients
3. The Chain Rule

<img src="./tangent_line.gif" alt="Venn Diagram" title="Venn Diagram" width="350" height="250">

This is a function f(x) that shows the differece between the secant line and tangent line. The secant line represents the average rate of change. The tangent line represents the instantaneous rate of change at a particular moment. This is where the idea of a derivatice comes into play. The derivative is the slope of a tangent line at a particular point and is defined as df/dx.

<img src="./partial_dif.png" alt="Venn Diagram" title="Venn Diagram" width="350" height="250">

The partial derivative is similar to the derivative. Say we have a function f(x,y). We now want to find the rate of change at a particular moment. Now since the function relies on x and y we must find the derivative for each of these independently.  Now we will need to find df/dx and df/dy, treating the varabile we arent differentiating as a constant

### What is the gradient then?
A gradient is nothing different then what we have talked about. A gradient is the vector of partial derivatives of a function. So the gradient of f(x,y) would be <df/dx, df/dy>. This vector can then show us the slope of a function in higher dimensions. Taking the positive gradient will point us in the direction of steepest ascent and the negative gradient in the direction of steepset descent. This is where the terminology "Gradient Descent" comes into play

### Now whats up with the chain rule?
<img src="./chain_rule.png" alt="Venn Diagram" title="Venn Diagram" width="750" height="250">

The chain rule will be used in pair with gradients. It allows us to differentiate variables that depend on other variables. This becomes one of the core reasons why the autograd engine works.
<br><br><br><br><br><br>


### Lets look at en example
Here we have a small DAG. The equation represented by the DAG is in the upper left hand corner.

<img src="./dag_final_final.png" alt="Venn Diagram" title="Venn Diagram" width="750" height="450">

We can then use the chain rule to calculate the partial derivative for each of these nodes with respect to F.

<img src="./dag_final_partial.png" alt="Venn Diagram" title="Venn Diagram" width="750" height="450">


Here is what that looks like in Python

In [1]:
from AutoPy import Tensor
A,B,C,D=Tensor(3),Tensor(5),Tensor(8),Tensor(2)

def add(x,y):
    return x + y

def multiply(x,y):
    return x * y

C = add(A,B)
F = multiply(C, D)
F.backward()

print(f"F grad: {F.grad}, C grad: {C.grad}, D grad: {D.grad}, A grad: {A.grad}, B grad: {B.grad}")

F grad: 1, C grad: 2, D grad: 8, A grad: 2.0, B grad: 2.0


### Let see how this translates to code in both Javascript

<div style="display: flex; align-items: center;">
    <img src="./javascript_code_snips/constructor.png" alt="JavaScript Constructor" title="JavaScript Constructor" width="600" height="250">
    <img src="./python_code_snips/constructor.png" alt="Python Constructor" title="Python Constructor" width="600" height="250">
</div>



<div style="display: flex; align-items: center;">
    <img src="./javascript_code_snips/mult_func.png" alt="JavaScript Constructor" title="JavaScript Constructor" width="600" height="450">
    <img src="./python_code_snips/mult_func.png" alt="Python Constructor" title="Python Constructor" width="600" height="450">
</div>

This represents the heart of the library. The autograd library is made up of many of the basic operations that make up most most such as addition, multiplication, etc. Lets take a look at 
- x1.grad += x2.grad * y.grad;

x1.grad += x2.grad represents the partial derivative of the loss with respect to x1. Since this is a multiplication operation, that means the partial derivative would be x2.

<br>
We the multiply it by y.grad which acts as the chain rule. y.grad will be accumulating all of the intermediary partial derivaitves we need. Simply multiplying by y.grad will be how we use the chain rule.

<br>
Finally, we accumulate the gradient using += since a variable can be used multiple times in a dag.


<div style="display: flex; align-items: center;">
    <img src="./javascript_code_snips/topo.png" alt="JavaScript Constructor" title="JavaScript Constructor" width="500" height="600">
    <img src="./python_code_snips/topo.png" alt="Python Constructor" title="Python Constructor" width="500" height="500">
</div>

This is the topological sort algorithm we use on the computational graph that runs in O(V+E)(verticies and edges) time. This sort the graph in a way that makes sure you have all the dependencies you need. A real world example would be sorting a DAG of classes and the prerequisites to make sure you take the classes in the right order.

<br>

You see at the end of the function we then reverse the sort. This is because originally it sorts the graph from left to right topologically. But since we want to start at the loss, the end, we must start at the end.

### Performance
I implemented 2 autograd libraries. One in Python and the other in Javascript. I also set up tests with Pytorch. I then tested each library on varying number of network paramters, calculating total floating points operations(FLOPS), to see how the autograd engine performed. Here were the results.

<img src="./performance_metrics/first_4_bar.png" alt="Python Constructor" title="Python Constructor" width="700" height="500">
<br>

<img src="./performance_metrics/last_4_bar.png" alt="Python Constructor" title="Python Constructor" width="700" height="500">

<br>

Pytorch was the fastest. For every trial it ran at around .3 seconds. However, the most shocking result was how performant Javascript was over Python seen here.

<img src="./performance_metrics/jscriptvspython.png" alt="Python Constructor" title="Python Constructor" width="900" height="600">

Where the most amount of time Javascript took was around 82 seconds and for Python, I had to extrapolate the seconds towards the end since it was taking to much time to run on my laptop.

### Conclusion

Javascript outperformed Python but a lot but did not come close to Pytorch. my next steps in evaluating Javascript as a contendor would be research and implementing these ideas
- WASM
    - wasm binaries can be big files
- Web Workers / Worker Threads(Node)
    - Heavy weight and not intended to be used in large numbers
    - No interaction with DOM
- V8/GC Code Optimization
    - Optimizations dependend on compiler version and browser